In [1]:
print('hello')

hello


In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

In [46]:
# HEADERS = {
#     'accept':'*/*',
#     'accept-encoding':'gzip, deflate, br, zstd',
#     'accept-language':'id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7'
# }

In [47]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Menjalankan browser di background
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-extensions")

In [48]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [49]:
# URL aplikasi Gojek di Google Play Store
url = "https://play.google.com/store/apps/details?id=com.gojek.app&hl=id"

try:
    # Buka URL
    driver.get(url)
    print("Halaman dibuka...")
    time.sleep(3)
    
    # Cari tombol "Lihat semua ulasan" dan klik
    try:
        see_all_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Lihat semua ulasan') or contains(text(), 'Baca ulasan') or contains(@aria-label, 'ulasan')]"))
        )
        see_all_reviews_button.click()
        print("Tombol ulasan diklik...")
        time.sleep(3)
    except TimeoutException as e:
        print(f"Tidak bisa menemukan tombol ulasan: {e}")
        # Coba cara lain - mungkin langsung ke URL ulasan
        try:
            review_url = f"{url}&showAllReviews=true"
            driver.get(review_url)
            print(f"Mencoba URL alternatif: {review_url}")
            time.sleep(3)
        except Exception as e2:
            print(f"Gagal mengakses URL alternatif: {e2}")
    
    # Data container
    reviews_data = []
    processed_reviews = set()  # Set untuk melacak ulasan yang sudah diproses
    
    # Fungsi untuk mengekstrak ulasan lebih banyak
    def extract_reviews_with_improved_scrolling(target_count=100, max_attempts=50):
        attempt = 0
        last_count = 0
        stagnant_count = 0
        
        while len(reviews_data) < target_count and attempt < max_attempts and stagnant_count < 5:
            attempt += 1
            
            # Tunggu beberapa saat agar halaman memuat lebih banyak ulasan
            time.sleep(5)
            
            # Cari semua elemen ulasan di halaman
            try:
                review_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'RHo1pe') or contains(@class, 'RHolpe')]")
                print(f"Percobaan {attempt}: Menemukan {len(review_elements)} elemen ulasan")
                
                # Ekstrak data dari elemen ulasan yang terlihat
                for review in review_elements:
                    try:
                        username_element = review.find_element(By.XPATH, ".//div[contains(@class, 'X5PpBb')]")
                        username = username_element.text.strip()
                        
                        review_text_element = review.find_element(By.XPATH, ".//div[contains(@class, 'h3YV2d')]")
                        review_text = review_text_element.text.strip()
                        
                        if username and review_text:
                            entry = {'user': username, 'review': review_text}
                            if entry not in reviews_data:
                                reviews_data.append(entry)
                                print(f"Ulasan baru: {username[:10]}... - {review_text[:20]}...")
                    except StaleElementReferenceException:
                        continue
                    except Exception as e:
                        print(f"Error saat mengekstrak data ulasan: {e}")
            
            except Exception as e:
                print(f"Error saat menemukan elemen ulasan: {e}")
            
            # Cek apakah jumlah ulasan bertambah
            if len(reviews_data) == last_count:
                stagnant_count += 1
                print(f"Tidak ada ulasan baru yang ditemukan ({stagnant_count}/5)")
            else:
                stagnant_count = 0  # Reset counter
            
            last_count = len(reviews_data)
            print(f"Total ulasan saat ini: {last_count}")
            
            # Lakukan scroll untuk menampilkan lebih banyak ulasan
            driver.execute_script("window.scrollBy(0, 1000);")  # Scroll lebih jauh lagi
            time.sleep(4)  # Tunggu setelah scroll

    # Jalankan fungsi ekstraksi
    print("Mulai ekstraksi ulasan...")
    extract_reviews_with_improved_scrolling(target_count=100, max_attempts=50)
    
    # Simpan data ke CSV
    csv_filename = "gojek_reviews.csv"
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['user', 'review']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in reviews_data:
            writer.writerow(data)
    
    print(f"Scraping selesai. {len(reviews_data)} ulasan disimpan ke {csv_filename}")

except Exception as e:
    print(f"Error utama: {e}")

finally:
    # Tutup browser
    driver.quit()

Halaman dibuka...
Tombol ulasan diklik...
Mulai ekstraksi ulasan...
Percobaan 1: Menemukan 20 elemen ulasan
Ulasan baru: prabandaru... - Ini aplikasi jangan ...
Ulasan baru: Jerry h... - Kalau ketentuan untu...
Ulasan baru: Desy Adela... - Pesan gosend sampai ...
Ulasan baru: Dewita Nur... - pesan makan udh seja...
Ulasan baru: Ridwan Mon... - Benar2 membantu. Cum...
Ulasan baru: Lin Wijaya... - Noted: Sudah ditangg...
Ulasan baru: fatma dewi... - Untuk pemesanan gofo...
Ulasan baru: Refika Sya... - Kecewa berkali2 sama...
Ulasan baru: Hifdzul Ta... - Aslinya saya puas ta...
Ulasan baru: Achie Dado... - Aplikasi sangat memb...
Ulasan baru: Ilham Mahe... - dalam aplikasi masih...
Ulasan baru: Hesti Viro... - Banyak banget jenis ...
Ulasan baru: rexq dio... - aplikasi mengalami b...
Ulasan baru: Syarif Ihz... - Kenapa saya turunin ...
Ulasan baru: amygos... - Sy pernah pesan goca...
Ulasan baru: Andy Putra... - Saya pesan makanan d...
Ulasan baru: Rin's Utam... - Hanya mau protes aja...
